In [1]:
import os

Reading API Keys

In [2]:
#Reading OpenAI Key
OPENAI_API_KEY=input("Enter OpenAI API Key")
#Reading Langchain API Key
LANGCHAIN_API_KEY=input("Enter Langchain API Key")

In [3]:
#Enabiling Langchain Tracing
LANGCHAIN_TRACING_V2= "true"
#OpenAI Model
OPENAI_MODEL="gpt-4o-mini"

In [ ]:
# installing langchain_openai package
! pip install langchain_openai

In [4]:
# initializing llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=OPENAI_MODEL,openai_api_key=OPENAI_API_KEY,temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11c490d70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11c492210>, root_client=<openai.OpenAI object at 0x107b81ac0>, root_async_client=<openai.AsyncOpenAI object at 0x11c490da0>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
# Installing Python package for SQLDatabase
! pip install langchain-community==0.2.9
! pip install sqlalchemy==2.0.28

In [5]:
#initializing Database object
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///baggage_tracking.db")
db

In [6]:
#creating Chain
from langchain.chains import create_sql_query_chain
generate_query = create_sql_query_chain(llm, db)
#invoking generate query chain to generate the SQL query
query = generate_query.invoke({"question":"How many passengers are there?","dialect":"sqllite","table_info":db.get_table_info()})
query

'SQLQuery: SELECT COUNT("passenger_id") AS "total_passengers" FROM "passenger_details";'

In [8]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.outputs import  Generation
class SQlExtractParser(BaseGenerationOutputParser[str]):
    """
    A parser for SQL output.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """
        Parse the SQL query input and return the extracted SQL query.

        Args:
            sql_query_input (str): The input SQL query string.
            partial (bool, optional): Whether the input is a partial query. Defaults to False.

        Returns:
            str: The extracted SQL query.
        """
        if not result:
            raise OutputParserException("Input SQL query is empty")
        
        print(f"Incoming query {result[0]}")

        if 'SQLQuery:' in result[0].text:
            sql_query = result[0].text.split('SQLQuery:', 1)[1].strip()
            print(f"Extracted SQL query: {sql_query}")
        else:
            sql_query = result[0].text

        return sql_query

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
def rephrase(user_query:str,query:str, query_result:str):
    answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
    )
    rephrase_answer = answer_prompt | llm | StrOutputParser()
    response=rephrase_answer.invoke({"question":user_query,"query":query,"result":query_result})
    print(f"Rephrased Response : %s", response)
    return response

In [11]:
ANSWER_PROMPTS = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
     Combine the the Question and SQL Result to generate a meaningful answer to the user.
     The answer should be in the format of a sentence.
     The answer should be a direct response to the user question.
     Respond to the question as an experienced airline ground staff

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

Chain No 1 :- query_chain. The query chain will run the query generator and send the output to SQlExtractParser() to extract the sql


In [9]:
generate_query = create_sql_query_chain(llm, db)
query_chain = generate_query | SQlExtractParser()

Chain No 2 :- execute_query. Executes the generated query on the DB using QuerySQLDataBaseTool(db=db)

In [10]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)

Chain No 3 :- answer_chain, Uses answer_prompt | llm | StrOutputParser() to generate an NLP for the DB query output

In [12]:
answer_chain = ANSWER_PROMPTS | llm | StrOutputParser()

Chaining everything together - Langchain provides us the option for chaining multiple chains together. We will create a final chain which will run all out three chain one after the othet

In [13]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
final_chain = (RunnablePassthrough.assign(query = query_chain).assign(result=itemgetter("query") | execute_query)) | answer_chain

Invoking the chain with user question and other parameters

In [14]:
answer=final_chain.invoke({"question":"How many passengers are there?","dialect":"sqllite","table_info":db.get_table_info()})

Incoming query text='SQLQuery: SELECT COUNT(DISTINCT "passenger_id") AS "total_passengers" FROM "passenger_details";'
Extracted SQL query: SELECT COUNT(DISTINCT "passenger_id") AS "total_passengers" FROM "passenger_details";


In [15]:
answer

'There are a total of 20 passengers.'